### Pass Map

In [2]:
# Import Libraries
import pandas as pd
from statsbombpy import sb

In [ ]:
matchesDF = sb.matches(competition_id=1267, season_id=107)
team = 'Nigeria'

# Get Nigeria matches id
nigeriaMatches = matchesDF[(matchesDF['home_team'] == team) |(matchesDF['away_team'] == team)]
matchID = nigeriaMatches['match_id'].to_list()

# Get the event data from all matches and store in a dataframe
events = []
for i in matchID:
    m_events = sb.events(i)
    m_events['match_id'] = i # Column for the matchid
    events.append(m_events)

# Join the dataframes
matchEvents = pd.concat(events, axis=0, ignore_index=True)

# Split columns
matchEvents[['location_x', 'location_y']] = matchEvents['location'].apply(pd.Series)
matchEvents[['pass_end_location_x', 'pass_end_location_y']] = matchEvents['pass_end_location'].apply(pd.Series)
matchEvents['shot_end_location'] = matchEvents['shot_end_location'].astype(str)
matchEvents[['shot_end_location_x', 'shot_end_location_y', 'shot_end_location_z']] = matchEvents['shot_end_location'].str.strip('[]').str.split(', ', expand=True)

Get passing events

In [9]:
passEvents = matchEvents[(matchEvents['type'] == 'Pass') & (matchEvents['team'] == team)]

# Filter for needed data
passMap = passEvents[['match_id','id','period','minute','player','player_id','position',
                'location_x','location_y','pass_recipient',
                'pass_end_location_x','pass_end_location_y','pass_type','pass_length',
                'pass_outcome','pass_goal_assist']]

# Applky the function and create a new column
def passoutcome(row):
    if pd.isna(row['pass_outcome']):
        return 'complete'
    elif row['pass_outcome'] == 'Incomplete':
        return 'incomplete'
    else:
        return None
    
passMap['pass_success'] = passMap.apply(passoutcome, axis = 1)

# Reset index
passMap.reset_index(drop=True, inplace=True)

# Assign assist
# # Assign assists
for i in range(len(passMap)):  # Start from 1 to avoid index error
    if (passMap.loc[i, 'pass_goal_assist'] == True):
        # Replace pass success to the assist
        passMap.at[i, 'pass_success'] = 'GoalAssist'

# Filter pass outcome for only incomplete and nan/complete passes
passMapDF = passMap[(passMap['pass_outcome'].isna() == True) | (passMap['pass_outcome'] == 'Incomplete')]

# Reset index and export
fullpassmap = passMapDF.reset_index(drop=True)
fullpassmap.to_csv(r'./PassMap.csv', index=False)

fullpassmap.head()

C:\Users\Razak\AppData\Local\Temp\ipykernel_20368\2756884658.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passMap['pass_success'] = passMap.apply(passoutcome, axis = 1)


,match_id,id,period,minute,player,player_id,position,location_x,location_y,pass_recipient,pass_end_location_x,pass_end_location_y,pass_type,pass_length,pass_outcome,pass_goal_assist,pass_success
0,3923881,72e4bb5b-f908-4af9-afc4-0824b4e0de96,1,0,Ademola Lookman,3447.0,Left Wing,61.0,40.1,William Troost-Ekong,35.1,52.5,Kick Off,28.715326,NaN,NaN,complete
1,3923881,da854004-2f92-4ff7-944b-a83198058536,1,0,William Troost-Ekong,5455.0,Center Back,35.1,52.5,Calvin Bassey Ughelumba,35.2,33.5,NaN,19.000263,NaN,NaN,complete
2,3923881,4465f67b-c718-43c2-a934-cae7d3b65c4d,1,0,Calvin Bassey Ughelumba,43779.0,Left Center Back,38.3,32.9,Oluwasemilogo Adesewo Ibidapo Ajayi,97.1,56.5,NaN,63.359290,NaN,NaN,complete
3,3923881,24e46dcf-9a75-4cbd-afac-c94e137494c6,1,0,Oluwasemilogo Adesewo Ibidapo Ajayi,3923.0,Right Center Back,97.1,56.5,Victor James Osimhen,112.7,49.8,NaN,16.977926,Incomplete,NaN,incomplete
4,3923881,e56ea592-671c-48cd-83d8-b463893e29ab,1,0,Zaidu Sanusi,31949.0,Left Wing Back,27.8,15.3,Alex Iwobi,33.8,26.2,Recovery,12.442266,NaN,NaN,complete


Get player list

In [70]:
# Get all player_id, player_name, jersey_number and country for all fixtures
players_list = []

for i in nigeriaMatches['match_id']:
    lineup = sb.lineups(match_id = i)[team]
    fullLineup = lineup[['player_id','player_name','jersey_number','country']]
    players_list.append(fullLineup)

players_df = pd.concat(players_list, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)

# Export players info
#players_df.to_csv(r'./PlayersList.csv', index=False)
players_df.head()

c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\s

,player_id,player_name,jersey_number,country
0,3139,Chidozie Collins Awaziem,20,Nigeria
1,3447,Ademola Lookman,18,Nigeria
2,3647,Alex Iwobi,17,Nigeria
3,3708,Kelechi Promise Iheanacho,14,Nigeria
4,3806,Bright Osayi-Samuel,12,Nigeria


To add position (Not needed in this case)

In [67]:
# Get all player_id, player_name, jersey_number and country for all fixtures
players_list = []

for i in nigeriaMatches['match_id']:
    lineup = sb.lineups(match_id = i)[team]

    lineup['player_position'] =  None

    for index, row in lineup.iterrows():
        positions = row['positions']
        if positions and len(positions) != 0:
            lineup.at[index, 'player_position'] = positions[0]['position']

    fullLineup = lineup[['player_id','player_name','jersey_number','country','player_position']]
    players_list.append(fullLineup)


players_df = pd.concat(players_list, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)

# Export players info
players_df.to_csv(r'./PlayersList.csv', index=False)
players_df.head()

c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\s

,player_id,player_name,jersey_number,country,player_position
0,3139,Chidozie Collins Awaziem,20,Nigeria,None
1,3447,Ademola Lookman,18,Nigeria,Left Wing
2,3647,Alex Iwobi,17,Nigeria,Left Defensive Midfield
3,3708,Kelechi Promise Iheanacho,14,Nigeria,Right Wing
4,3806,Bright Osayi-Samuel,12,Nigeria,None
